In [12]:
import numpy as np
import chess
import json
import timeit

In [9]:
def getNormX(position):
    return (position%8.0)/7.0
def getNormY(position):
    return float(position/8)/7.0

In [10]:
def calcGlobalFeatures(board):
    #side2Move
    #THIS IS PASSED DIRECTLY TO SECOND LAYER
    side2Move = float(board.turn)
    
    #king positions (this is covered elsewhere, but in a different group)
    #THIS IS PASSED DIRECTLY TO SECOND LAYER
    kingPosition = [getNormX(board.king(chess.WHITE)), 
        getNormY(board.king(chess.WHITE)), 
        getNormX(board.king(chess.BLACK)),
        getNormY(board.king(chess.BLACK))]
    
    #castlingRights
    #THIS IS PASSED DIRECTLY TO SECOND LAYER (I think)
    castlingRights = [float(board.has_kingside_castling_rights(chess.WHITE)), 
        float(board.has_queenside_castling_rights(chess.WHITE)), 
        float(board.has_kingside_castling_rights(chess.BLACK)), 
        float(board.has_queenside_castling_rights(chess.BLACK))]
    
    #numOfEachPieceType 
    #THIS IS PASSED DIRECTLY TO SECOND LAYER
    numOfEachPieceType = [len(board.pieces(chess.PAWN, chess.WHITE))/8.0, 
        len(board.pieces(chess.KNIGHT, chess.WHITE))/2.0,
        len(board.pieces(chess.BISHOP, chess.WHITE))/2.0,
        len(board.pieces(chess.ROOK, chess.WHITE))/2.0,
        len(board.pieces(chess.QUEEN, chess.WHITE))/1.0,
        len(board.pieces(chess.PAWN, chess.BLACK))/8.0, 
        len(board.pieces(chess.KNIGHT, chess.BLACK))/2.0,
        len(board.pieces(chess.BISHOP, chess.BLACK))/2.0,
        len(board.pieces(chess.ROOK, chess.BLACK))/2.0,
        len(board.pieces(chess.QUEEN, chess.BLACK))/1.0]

In [42]:
#see variable SEE::SEE_MAT for values (WK is white king i think)
#NormalizeCount(SEE::SEE_MAT[WK] + SEE::SEE_MAT[WK] / 2 - SEE::SEE_MAT[whitePt], SEE::SEE_MAT[WK] * 2)
'''
static const Score SEE_MAT[14] = {
    1500, // WK
    975, // WQ
    500, // WR
    325, // WN
    325, // WB
    100, // WP
}
'''

def getLowestValueAttackerScore(board, position, color):
    attackers = board.attackers(color, position)
    #determine the lowest value attacker
    if not bool(attackers): #no attackers
        value = 2250.0 #makes the score 0
    elif bool(attackers & board.pieces(chess.PAWN, color)):
        value = 100.0
    elif bool(attackers & board.pieces(chess.BISHOP, color)):
        value = 325.0
    elif bool(attackers & board.pieces(chess.KNIGHT, color)):
        value = 325.0
    elif bool(attackers & board.pieces(chess.ROOK, color)):
        value = 500.0
    elif bool(attackers & board.pieces(chess.QUEEN, color)):
        value = 975.0
    else: #Must be a king attacker
        value = 1500.0
    return (2250.0 - value)/3000.0

In [43]:
def getSinglePieceFeatures(board, position, exists=True):
    #return array of pieceExists, XYposition, and attackers and defenders
    if exists:
        whiteAttackerScore = getLowestValueAttackerScore(board, 
                                 position, chess.WHITE)
        blackAttackerScore = getLowestValueAttackerScore(board, 
                                 position, chess.BLACK)
        feats = [1.0, getNormX(position), getNormY(position), 
                    whiteAttackerScore, blackAttackerScore]
    else:
        feats = [0.0, 0.0, 0.0, 0.0, 0.0]
        
def getPawnFeatures(board, color):
    pawns = board.pieces(chess.PAWN, color)
    #Assign each pawn to a slot based on X position
    slots = []
    unassignedSlots = []
    unassignedPieces = []
    for x in range(8):
        column = pawns & chess.BB_FILES[x]
        if len(column) == 1: #exactly one pawn in column
            position = column.pop()
            slots.append(getSinglePieceFeatures(board, position))
        elif len(column) == 0: #no pawns in column
            slots.append(None)
            unassignedSlots.append(x)
        else: #more than one pawn in column
            position = column.pop()
            slots.append(getSinglePieceFeatures(board, position))
            unassignedPieces.extend(list(column))
    #match unpaired slots to pawns 
    for x in unassignedSlots:
        #make sure there are still extra pawns
        if len(unassignedPieces) == 0:
            break
        #find closest pawn position
        position = min(unassignedPieces, key=lambda pos : abs(x-pos%8))
        slots[x] = getSinglePieceFeatures(position)
        unassignedPieces.remove(position)
    #any extra pawns after this don't have a slot and are ignored
    #combine slots into a single array and return it
    return [feature for slot in slots for feature in slot]

def getPairPieceFeatures(board, pieceType, color):
    #for rooks, bishops, and knights, there are only 2 slots
    pieces = board.pieces(pieceType, color)
    if len(pieces) == 0:
        feats = getSinglePieceFeatures(None, None, exisits=False)*2
    elif len(pieces) == 1:
        position = pieces.pop()
        emptyFeatures = getSinglePieceFeatures(None, None, exisits=False)
        existingFeatures = getSinglePieceFeatures(board, position)
        if position%8 <4:
            feats = existingFeatures + emptyFeatures
        else:
            feats = emptyFeatures + existingFeatures
    else:
        position1 = pieces.pop()
        position2 = pieces.pop()
        feats1 = getSinglePieceFeatures(board, position1)
        feats2 = getSinglePieceFeatures(board, position2)
        if position1%8 < position2%8:
            feats = feats1 + feats2
        else:
            feats = feats2 + feats1
    return feats
    
def getQueenFeatures(board, color):
    queens = board.pieces(pieceType, color)
    if len(queens) > 0:
        position = queens.pop()
        feats = getSinglePieceFeatures(board, position)
    else:
        feats = getSinglePieceFeatures(None, None, exists=False)
    
def getKingFeatures(board, color): #maybe not needed
    kings = board.pieces(pieceType, color)
    if len(kingss) > 0:
        position = queens.pop()
        feats = getSinglePieceFeatures(board, position)
    else: #this should never happen
        feats = getSinglePieceFeatures(None, None, exists=False)
    

#the main function for this cell:
def calcPieceFeatures(board):
    pairPieces = [chess.ROOK, chess.BISHOP, chess.KNIGHT]
    pairPiecesFeatures = []
    for pieceType in pairPieces:
        pairPiecesFeatures.append(getPairPieceFeatures(board, pieceType, chess.WHITE))
        pairPiecesFeatures.append(getPairPieceFeatures(board, pieceType, chess.BLACK))
    pieceFeatures = (getPawnFeatures(board, chess.WHITE)
                     +getPawnFeatures(board, chess.BLACK),
                     +pairPiecesFeaturesWhite,
                     +pairPiecesFeaturesBlack, 
                     +getQueenFeatures(board, chess.WHITE), 
                     +getQueenFeatures(board, chess.BLACK),
                     +getKingFeatures(board, chess.WHITE), 
                     +getKingFeatures(board, chess.BLACK))
    
    return pieceFeatures

In [44]:
def calcSquareFeatures(board):
    feats = []
    for position in range(64):
        feats.append(getLowestValueAttackerScore(board, position, chess.WHITE))
        feats.append(getLowestValueAttackerScore(board, position, chess.BLACK))
    return feats

In [45]:
def fen2Features(fen):
    board = chess.Board(fen)
    
    globalFeatures = calcGlobalFeatures(board)
    
    pieceFeatures = calcPieceFeatures(board)
    
    squareFeatures = calcSquareFeatures(board)
    
    return {'globalFeatures':globalFeatures, 
            'pieceFeatures':pieceFeatures, 
            'squareFeatures':squareFeatures}



In [46]:
#this takes the raw data and preprocesses it into features
#open the raw data
path = '/home/rvansoelen/rbmcData/isc_games/'
files = ['games1.json', 'games2.json', 'games3.json', 'games4.json']
start = timeit.timeit()
for file in files:
    filepath = path+file
    with open(filepath, 'r') as fid:
        dataset = []
        for line in fid:
            game = json.loads(line)
            if game['status'] != 'complete':
                print('Skipping game ', game['id'], ', status is ', game['status'])
                continue
            for turn in game['gameHistory']:
                fen = turn['truth']['fen']+' '+str(turn['moveNumber'])
                boardFeatures = fen2Features(fen)
                print boardFeatures
                break
                
            break
    break
    
print('Done')
print(timeit.timeit()-start)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [31]:
game['gameHistory'][0]['moveNumber']

0

In [ ]:
#get the boards and moves from each board

#augement data: (does this bias the representation of states??)
	#perform a time limited search on board states from raw data
	#randomly sample from this search to get dataset
	#label board state with best move, either from stockfish, raw data, or other person's bot



In [ ]:
#partition data into test, validation, and train


#randomly sort and save samples